In [1]:
%pip install sophuspy

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 10.3 MB/s eta 0:00:0000:0100:01
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Created wheel for sophuspy: filename=sophuspy-1.2.0-cp312-cp312-macosx_14_0_arm64.whl size=197019 sha256=f453c8c1750d4ecc8c36168ccd5be010027967ca6ae4d18fe494046f2ca1bb0a
  Stored in directory: /Users/aj/Library/Caches/pip/wheels/27/0d/47/559ce5edf9b42cfc2aa2ba78f0e9df60f20b2eb642c0689461
Successfully built sophuspy

[notice] A new release of pip is available: 24.1.2 -> 24.2
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [1]:
import numpy as np
import sophuspy as sp
from scipy.spatial.transform import Rotation as R

In [4]:
r = R.from_rotvec(np.pi/2 * np.array([0, 0, 1])) # 90 degreee rotation along the z-axis

rSP = sp.SO3(r.as_matrix())
print(rSP)

SO3([[2.22044604925031e-16,                   -1,                    0],
     [                   1, 2.22044604925031e-16,                    0],
     [                   0,                    0,                    1]])


In [14]:
# use logarithmic map to get the lie algebra
so3 = rSP.log()
print(so3)

# use the `hat` method to convert to a skew-symmetric matrix
so3sm = sp.SO3.hat(so3)
print(so3sm)

# convert skew-symmetric matrix to a vector using `vee`
# vee = sp.SO3.vee(so3sm) <-- The `vee` operator doesn't exist, will update this via a pull request to https://github.com/craigstar/SophusPy

# update by perturbation model 

[0.         0.         1.57079633]
[[ 0.         -1.57079633  0.        ]
 [ 1.57079633  0.         -0.        ]
 [-0.          0.          0.        ]]
